In [25]:
#Importing the libraries
from pprint import pprint
import requests
import http.client
import json
import pandas as pd
from datetime import datetime as dt

In [26]:
# Creating the API call
conn = http.client.HTTPSConnection("www.jambase.com")

headers = { 'Accept': "application/json" }

conn.request("GET", "/jb-api/v1/events?geoCityId=jambase%3A4219129&apikey=964b3dbc-0ea0-46b2-bbf9-476d3cf9ab18", headers=headers)

res = conn.getresponse()
data = res.read()

decoded_data = data.decode("utf-8")

# Use the json module to load the JSON data
json_data = json.loads(decoded_data)

# Print the JSON data using pprint (optional)
from pprint import pprint
pprint(json_data)

# Export the JSON data to a file (e.g., 'output.json')
with open('output.json', 'w') as json_file:
    json.dump(json_data, json_file)

# Close the connection
conn.close()

{'events': [{'@type': 'Concert',
             'dateModified': '2023-10-31T16:10:08Z',
             'datePublished': '2023-08-16T14:27:33Z',
             'doorTime': '',
             'endDate': '2023-11-21',
             'eventAttendanceMode': 'live',
             'eventStatus': 'scheduled',
             'identifier': 'jambase:10892778',
             'image': 'https://www.jambase.com/wp-content/uploads/2017/02/corb-lund-corb-lund-153ccac4-e771-496f-8b62-6d3dd96610f0_285371_TABLET_LANDSCAPE_16_9-1480x832.jpg',
             'isAccessibleForFree': False,
             'location': {'@type': 'Venue',
                          'address': {'@type': 'PostalAddress',
                                      'addressCountry': {'@type': 'Country',
                                                         'alternateName': 'USA',
                                                         'identifier': 'US',
                                                         'name': 'United '
                         

In [27]:
# Checking the API pull for the event names
n=0
for i in range(0,len(json_data['events'])):
    n+=1
    print(f"{n}: {json_data['events'][i]['name']}")

1: Corb Lund at Sam’s Burger Joint
2: DeeBaby at Paper Tiger
3: Intocable at Majestic Theatre
4: Anything Box at Sam’s Burger Joint
5: Purple Kiss at Vibes Event Center
6: Jonny Craig at The Rock Box
7: Pussy Riot at Paper Tiger
8: Slaid Cleaves at Sam’s Burger Joint
9: Kidz Bop at Majestic Theatre
10: Wild Party at Paper Tiger
11: Patricia Vonne at Sam’s Burger Joint
12: Cancamusa at Paper Tiger
13: Donovan Keith at Sam’s Burger Joint
14: Brit Floyd at The Aztec Theatre
15: Angel Du$t at Paper Tiger
16: Andrés at Vibes Event Center
17: INOHA at Sam’s Burger Joint
18: Andrea Bocelli at Frost Bank Center
19: Jeff Rosenstock at Paper Tiger
20: Jaret Ray Reddick at Sam’s Burger Joint
21: We Are Scientists at Paper Tiger
22: Nascar Aloe at Studio 210 Nightclub and Bar
23: The Meteors at Vibes Event Center
24: Jason Eady at Sam’s Burger Joint
25: Frozen Soul at The Rock Box
26: Lauren Daigle at Frost Bank Center
27: Brett Young at Cowboys Dancehall
28: Suffocation at Paper Tiger
29: Prayers

In [28]:
#Parsing the event info

# Creating empty lists to append parsed information to.
headliner_name=[]
opener_name=[]
artist_genre=[]
event_date=[]
event_venue=[]
event_status=[]
venue_address=[]
ticket_link=[]
# Define the condition (e.g., if the value is positive)
condition = lambda x: x is True


# Looping through events and extracting relevant info.
for event in range(0,len(json_data['events'])):

    #Skipping non-concert events
    if json_data['events'][event]['@type']!='Concert':
        pass
    else:

        #grabbing the headliner's name
        for artist in range(len(json_data['events'][event]['performer'])):
            if json_data['events'][event]['performer'][artist]['x-isHeadliner'] is True:
                headliner_name.append(json_data['events'][event]['performer'][artist]['name'])
            else:
                pass
            
        event_date.append(json_data['events'][event]['endDate'])
        event_venue.append(json_data['events'][event]['location']['name'])
        event_status.append(json_data['events'][event]['eventStatus'])
        #Creating a composite address field
        venue_address.append(json_data['events'][event]['location']['address']['streetAddress']+', '+ 
                             json_data['events'][event]['location']['address']['addressLocality']+' '+
                             json_data['events'][event]['location']['address']['addressRegion']['name']+' '+
                             json_data['events'][event]['location']['address']['postalCode'])
        # We are assigning genre based on the headliner
        artist_genre.append(json_data['events'][event]['performer'][0]['genre'])
        # We are only taking the first ticketing url here
        ticket_link.append(json_data['events'][event]['offers'][0]['url'])

In [29]:
print(len(headliner_name), len(opener_name), len(artist_genre), len(event_date), len(event_status), len(event_venue), len(venue_address))


40 0 40 40 40 40 40


In [30]:
print(headliner_name, opener_name)

['Corb Lund', 'DeeBaby', 'Intocable', 'Anything Box', 'Purple Kiss', 'Jonny Craig', 'Pussy Riot', 'Slaid Cleaves', 'Kidz Bop', 'Wild Party', 'Patricia Vonne', 'Cancamusa', 'Donovan Keith', 'Brit Floyd', 'Angel Du$t', 'Andrés', 'INOHA', 'Andrea Bocelli', 'Jeff Rosenstock', 'Jaret Ray Reddick', 'We Are Scientists', 'Nascar Aloe', 'The Meteors', 'Jason Eady', 'Frozen Soul', 'Lauren Daigle', 'Brett Young', 'Suffocation', 'Prayers', 'Texas Flood – a Tribute to Stevie Ray Vaughan', 'Mariachi Nuevo Tecalitlan', 'Tommy Emmanuel', 'Mark Searcy', 'Blood Club', 'Devora', 'Marie Osmond', 'Silverstein', 'Early Moods', 'Jarekus Singleton', 'Missio'] []


In [31]:
# Adding an 'up-to-date as of' field
last_run=dt.today().date()

#Creating the structured pandas dataset
all_concerts=pd.DataFrame({'headliner_name':headliner_name,
                           
                           'genre': artist_genre, 
                           'event_date':event_date,
                           'event_status':event_status,
                           'event_venue_nm':event_venue, 
                           'venue_address':venue_address, 
                           'up_to_date_as_of':last_run})


all_concerts.head()

,headliner_name,genre,event_date,event_status,event_venue_nm,venue_address,up_to_date_as_of
0,Corb Lund,"[americana, country-music, folk, rock]",2023-11-21,scheduled,Sam’s Burger Joint,"330 East Grayson Street, San Antonio Texas 78215",2023-11-21
1,DeeBaby,[],2023-11-21,scheduled,Paper Tiger,"2410 N St Mary's St, San Antonio Texas 78212",2023-11-21
2,Intocable,"[latin, pop]",2023-11-22,scheduled,Majestic Theatre,"226 East Houston St., San Antonio Texas 78205",2023-11-21
3,Anything Box,[],2023-11-22,scheduled,Sam’s Burger Joint,"330 East Grayson Street, San Antonio Texas 78215",2023-11-21
4,Purple Kiss,[pop],2023-11-24,scheduled,Vibes Event Center,"1211 East Houston, San Antonio Texas 78205",2023-11-21


In [63]:
#pulling artist info for the first 5 artists
headliners=headliner_name[0:5]

In [66]:
# Last.fm API pull
import requests

last_fm_api='9e4e0b01334e763557e0ae937759c950'
listeners=[]
playcounts=[]

for artist in headliners:
    artist_name=artist
    # Last.fm API endpoint for user.getTopTracks
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={last_fm_api}&format=json"


    # Making the GET request
    response = requests.get(url).json()
    listeners.append(response['artist']['stats']['listeners'])
    playcounts.append(response['artist']['stats']['playcount'])


In [67]:
playcounts

['1189759', '171385', '2071098', '394754', '10789412']

Consider pulling additional data from Itunes
Can you look at historical data
can you add multiple cities
look at descriptives statistics
Consider looking at Reddit to extend the data.

Add very basic analysis as well.
Think about ways to enrich the data and add some more information.

# Setlist.fm API pull

In [86]:
print(response['artist']['bio'].keys())

dict_keys(['links', 'published', 'summary', 'content'])


In [89]:
pprint(response['artist']['bio']['summary'])

('PURPLE KISS (퍼플키스; also stylized as PURPLE K!SS) is a six-member girl group '
 'under RBW. Before their official debut, the group released two digital '
 'pre-release singles: "My Heart Skip a Beat" and "Can We Talk Again". '
 'Originally as seven, they made their official debut on March 15, 2021 with '
 'the mini album Into Violet, and its respective title track "Ponzona".\n'
 '\n'
 'On September 8, 2021, they relased their 2nd mini album entitled Hide & Seek '
 'with the title track Zombie <a '
 'href="https://www.last.fm/music/Purple+Kiss">Read more on Last.fm</a>')


In [92]:
# Getting album information:

last_fm_api='9e4e0b01334e763557e0ae937759c950'

artist_name='The National'
# Last.fm API endpoint for user.getTopTracks

url = f"http://ws.audioscrobbler.com/2.0/?method=artist.gettopalbums&artist={artist_name}&api_key={last_fm_api}&format=json"


# Making the GET request
response = requests.get(url).json()
pprint(response)

{'topalbums': {'@attr': {'artist': 'The National',
                         'page': '1',
                         'perPage': '50',
                         'total': '38042',
                         'totalPages': '761'},
               'album': [{'artist': {'mbid': '664c3e0e-42d8-48c1-b209-1efca19c0325',
                                     'name': 'The National',
                                     'url': 'https://www.last.fm/music/The+National'},
                          'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/1718c111d11bce147b42e6ec6b922414.png',
                                     'size': 'small'},
                                    {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/1718c111d11bce147b42e6ec6b922414.png',
                                     'size': 'medium'},
                                    {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/1718c111d11bce147b42e6ec6b922414.png',
                                     'size': 'larg

In [112]:
# need to extract the mbids to then search for the albums
mbids = []
album_names = []
if 'topalbums' in response and 'album' in response['topalbums']:
    for i in range(len(response['topalbums']['album'])):
        album_data = response['topalbums']['album'][i]
        mbid = album_data.get('mbid', None)  # Use get to handle missing 'mbid'
        name = album_data.get('name', None)  # Use get to handle missing 'name'

        # Append to lists only if both 'mbid' and 'name' are present
        if mbid is not None and name is not None:
            mbids.append(mbid)
            album_names.append(name)
        else:
            pass
else:
    print("No 'topalbums' or 'album' found in the response.")


In [113]:
mbids

['9bce96cc-8d4f-38f3-97d5-decb81ab7119', 'd4e57d35-d8f8-4302-bca4-534b6227e284', 'daaa313f-2f30-39e7-a8f0-511276fc7341', '361eb780-0b77-479f-bf80-9f2ad813d3e0', '75abcec3-1432-3b63-ba0a-9e49279e314f', '04fd79dc-6e18-4982-9192-3edd0145c257', '0c38b4ef-ad05-44e7-bcc6-8da8ad6f28df', '2838cd2c-6f07-4633-b0c3-2e724beda5c6', '175b51a8-7ad4-466f-a117-1c28ceb722e2', '23b46f57-8f10-4338-9dcb-a6cbf7713709', '84a18ccc-7b2a-4924-acf4-539811c092d2', 'a899d853-7427-4162-bc3a-8e133ab83837', '173c8136-de0c-4a85-a46d-9b16c8b0e9ee', '724a7161-52b4-4689-96dd-a5170810593b', '39145749-3bd3-436e-bfbe-260928f21aa6']

In [114]:
for id in mbids:
    last_fm_api='9e4e0b01334e763557e0ae937759c950'

    # Last.fm API endpoint for user.getTopTracks

    url = f"http://ws.audioscrobbler.com/2.0/?method=album.getinfo&mbid={id}&api_key={last_fm_api}&format=json"


    # Making the GET request
    response = requests.get(url).json()
    pprint(response)

{'album': {'artist': 'The National',
           'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/1718c111d11bce147b42e6ec6b922414.jpg',
                      'size': 'small'},
                     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/1718c111d11bce147b42e6ec6b922414.jpg',
                      'size': 'medium'},
                     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/1718c111d11bce147b42e6ec6b922414.jpg',
                      'size': 'large'},
                     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/1718c111d11bce147b42e6ec6b922414.jpg',
                      'size': 'extralarge'},
                     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/1718c111d11bce147b42e6ec6b922414.jpg',
                      'size': 'mega'},
                     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/1718c111d11bce147b42e6ec6b922414.jpg',
                      'size': ''}],
           'listeners': '658378